In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import load_breast_cancer, load_digits, load_iris
from sklearn.metrics import mean_squared_error

from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder

from category_encoders import OneHotEncoder

from sklearn.pipeline import make_pipeline
import sklearn
sklearn.set_config(display = 'diagram')
from sklearn.feature_selection import RFE

In [82]:
df = pd.read_csv('../datasets/train.csv')
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [85]:
new_df = df.drop(['Alley', 'Lot Frontage', 'Pool QC', 'Fence', 'Misc Feature', 'Fireplace Qu'], axis = 1)

In [86]:
new_df = new_df.dropna()

In [87]:
ohe = OneHotEncoder()

In [89]:
X = new_df.drop(['SalePrice'], axis = 1)
y = new_df['SalePrice']

In [90]:
ohe = OneHotEncoder(use_cat_names = True)

In [91]:
X_encoded = ohe.fit_transform(X)

C:\Users\Lindsey\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [92]:
X_encoded.shape

(1866, 263)

In [93]:
y.shape

(1866,)

In [97]:
lr = LinearRegression()
lr.fit(X_encoded, y)

LinearRegression()

In [100]:
model = sm.OLS(y, X_encoded).fit()

In [104]:
model.pvalues.sort_values()

Misc Val                6.139595e-38
Overall Qual            8.076899e-15
Gr Liv Area             5.270042e-13
Overall Cond            3.678649e-12
Lot Area                4.100512e-11
Total Bsmt SF           2.497506e-09
Screen Porch            3.552355e-09
BsmtFin SF 1            6.371309e-09
2nd Flr SF              1.933799e-07
Mas Vnr Area            1.947988e-04
Year Built              2.164902e-04
Neighborhood_GrnHill    6.881962e-04
1st Flr SF              3.753349e-03
Kitchen AbvGr           6.702703e-03
Wood Deck SF            8.688364e-03
Fireplaces              1.888019e-02
Garage Area             2.372150e-02
Garage Cars             3.625214e-02
Pool Area               4.347453e-02
BsmtFin SF 2            4.530331e-02
Bsmt Full Bath          5.460725e-02
Full Bath               5.676535e-02
Neighborhood_StoneBr    8.207077e-02
Bedroom AbvGr           9.669958e-02
Roof Matl_Membran       1.047648e-01
Condition 2_RRAe        1.474329e-01
Roof Matl_WdShake       1.498082e-01
B

In [3]:
df.corr()['SalePrice'].sort_values(ascending=False)

SalePrice          1.000000
Overall Qual       0.800207
Gr Liv Area        0.697038
Garage Area        0.650270
Garage Cars        0.648220
Total Bsmt SF      0.628925
1st Flr SF         0.618486
Year Built         0.571849
Year Remod/Add     0.550370
Full Bath          0.537969
Garage Yr Blt      0.533922
Mas Vnr Area       0.512230
TotRms AbvGrd      0.504014
Fireplaces         0.471093
BsmtFin SF 1       0.423519
Lot Frontage       0.341842
Open Porch SF      0.333476
Wood Deck SF       0.326490
Lot Area           0.296566
Bsmt Full Bath     0.283662
Half Bath          0.283001
2nd Flr SF         0.248452
Bsmt Unf SF        0.190210
Bedroom AbvGr      0.137067
Screen Porch       0.134581
3Ssn Porch         0.048732
Mo Sold            0.032735
Pool Area          0.023106
BsmtFin SF 2       0.016255
Misc Val          -0.007375
Yr Sold           -0.015203
Low Qual Fin SF   -0.041594
Bsmt Half Bath    -0.045328
Id                -0.051398
MS SubClass       -0.087335
Overall Cond      -0

In [4]:
df.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


In [5]:
pd.options.display.max_rows = None

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [6]:
df['Lot Shape'].value_counts()

Reg    1295
IR1     692
IR2      55
IR3       9
Name: Lot Shape, dtype: int64

In [7]:
df['Bsmt Qual'] = df['Bsmt Qual'].map({np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5})
df['Lot Shape'] = df['Lot Shape'].map({'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3})

In [8]:
df['Lot Shape'].value_counts()

3    1295
2     692
1      55
0       9
Name: Lot Shape, dtype: int64

In [9]:
df = df[df['Total Bsmt SF'].notna()]
df = df[df['Garage Area'].notna()]

In [29]:
X = df[['Overall Qual', 'Total Bsmt SF', 'Gr Liv Area', 'Garage Area', 'Year Built', 'Roof Matl', 'Land Contour', 'Neighborhood', 'Bsmt Qual']]

In [30]:
y = df['SalePrice']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [32]:
pipe = make_pipeline(OneHotEncoder(use_cat_names = True), PolynomialFeatures(), StandardScaler(), Ridge())
pipe

Pipeline(steps=[('onehotencoder', OneHotEncoder(use_cat_names=True)),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [33]:
params = {
    'polynomialfeatures__interaction_only': [True, False],
    'ridge__alpha': [.1, 1, 10, 100]   
}

In [34]:
gs = GridSearchCV(pipe, params, n_jobs = -1) 

In [35]:
    gs.fit(X_train, y_train)

C:\Users\Lindsey\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


GridSearchCV(estimator=Pipeline(steps=[('onehotencoder',
                                        OneHotEncoder(use_cat_names=True)),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             n_jobs=-1,
             param_grid={'polynomialfeatures__interaction_only': [True, False],
                         'ridge__alpha': [0.1, 1, 10, 100]})

In [17]:
gs.best_params_

{'polynomialfeatures__interaction_only': True, 'ridge__alpha': 100}

In [36]:
pd.DataFrame(gs.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_polynomialfeatures__interaction_only,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.500914,0.015503,0.051568,0.004542,True,100,"{'polynomialfeatures__interaction_only': True,...",0.902579,0.676584,0.862756,0.880281,0.903657,0.845171,0.085650,1
7,0.506310,0.022298,0.051771,0.005378,False,100,{'polynomialfeatures__interaction_only': False...,0.903210,0.640542,0.864872,0.881060,0.906052,0.839147,0.100442,2
2,0.538693,0.046264,0.058968,0.010501,True,10,"{'polynomialfeatures__interaction_only': True,...",0.902074,0.511359,0.845274,0.883285,0.901324,0.808663,0.150073,3
6,0.506510,0.034553,0.052171,0.004115,False,10,{'polynomialfeatures__interaction_only': False...,0.902440,0.476510,0.849019,0.884725,0.904010,0.803341,0.164611,4
5,0.587464,0.060600,0.055769,0.003654,False,1,{'polynomialfeatures__interaction_only': False...,0.877988,0.365592,0.823743,0.877734,0.893636,0.767738,0.202466,5
1,0.526698,0.032473,0.054569,0.008254,True,1,"{'polynomialfeatures__interaction_only': True,...",0.878415,0.377583,0.815987,0.873466,0.889518,0.766994,0.196369,6
4,0.580469,0.096859,0.083352,0.016454,False,0.1,{'polynomialfeatures__interaction_only': False...,0.845630,0.301101,0.798797,0.861367,0.888142,0.739007,0.220870,7
0,0.551285,0.053019,0.058568,0.008424,True,0.1,"{'polynomialfeatures__interaction_only': True,...",0.841472,0.293261,0.787475,0.852405,0.882388,0.731400,0.221208,8


In [18]:
pipe = make_pipeline(OneHotEncoder(use_cat_names = True), PolynomialFeatures(interaction_only = True), StandardScaler(), Ridge(alpha = 100))

In [19]:
pipe.fit(X_train, y_train)

C:\Users\Lindsey\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Roof Matl', 'Land Contour',
                                     'Neighborhood'],
                               use_cat_names=True)),
                ('polynomialfeatures',
                 PolynomialFeatures(interaction_only=True)),
                ('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=100))])

In [20]:
pipe.score(X_train, y_train)

0.9191623845774859

In [21]:
pipe.score(X_test, y_test)

0.733751867917238

In [22]:
preds = pipe.predict(X_test)

In [23]:
mean_squared_error(preds, y_test, squared = False)

38313.2214170409

In [24]:
pipe.fit(X, y)

C:\Users\Lindsey\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Roof Matl', 'Land Contour',
                                     'Neighborhood'],
                               use_cat_names=True)),
                ('polynomialfeatures',
                 PolynomialFeatures(interaction_only=True)),
                ('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=100))])

In [25]:
pipe.score(X, y)

0.9127076098772009

In [ ]:
cross_val_score(X, y)

In [26]:
preds = pipe.predict(X)

In [27]:
mean_squared_error(preds, y, squared = False)

23412.502421278437

Model 3 = 24087 on X, y<br>
26961 on X_train, y_train

In [ ]:
df_test = pd.read_csv('../datasets/test.csv')
df_test['Bsmt Qual'] = df_test['Bsmt Qual'].map({np.nan: 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5})
df_test.head()

In [ ]:
X_test = df_test[['Overall Qual', 'Total Bsmt SF', 'Gr Liv Area', 'Year Built', 'Garage Area', 'Roof Matl', 'Land Contour', 'Neighborhood', 'Bsmt Qual']]
X_test.shape

In [ ]:
predictions = pipe.predict(X_test)
predictions.shape

In [ ]:
df_sales = pd.DataFrame(predictions, columns = ['SalePrice'])
df_sales.shape

In [ ]:
df_model4 = pd.concat([df_test['Id'], df_sales], axis = 1)
df_model4.info()

In [ ]:
df_model4.set_index('Id', inplace = True)

In [ ]:
df_model4.info()

In [ ]:
df_model4.to_csv('../submissions/model5_submission.csv')

Used: 'Overall Qual', 'Total Bsmt SF', 'Gr Liv Area', 'Year Built', 'Garage Area', 'Roof Matl', 'Land Contour', 'Neighborhood', 'Bsmt Qual'<br>
Converted 'Bsmt Qual' to 0-5

pipe = make_pipeline(OneHotEncoder(use_cat_names = True), PolynomialFeatures(interaction_only = True), StandardScaler(), Ridge(alpha = 100))